<a href="https://colab.research.google.com/github/jharilal/candlestick_analysis/blob/main/Candlestick_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Candlestick Indicator Analysis



## Outline



1. Problem Questions
2. Project Installs
3. Data Collection
4. Data Preparation
5. Data Analysis
6. Report

## Problem Questions

 - How accurate is the hammer technical indicators in predicting stock price movement for the top 100 / 500 companies over the past 5 years?
 - Does the industry / sub industry demonstrate a different amount of success with the hammer?
 - Whaat's the rate of success for the hammer?
 - What measurable difference is there between a hammer on a downtrend and uptrend?
 - How often do these indicators occur?

 These questions help answer the measurable reliability of using the hammer to guide trading decisions and could add insight for potential measure of success.


## Project Installs

In [1]:
!pip install yahoofinance
!pip install yfinance

     |████████████████████████████████| 63 kB 1.2 MB/s 
     |████████████████████████████████| 6.4 MB 9.9 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
import yfinance as yf
import pandas as pd
msft_tick = yf.download('MSFT')

[*********************100%***********************]  1 of 1 completed


## Data Collection

In [3]:
nasdaq_df = pd.read_html('https://en.wikipedia.org/wiki/Nasdaq-100')

In [4]:
nasdaq_df[3]

,Company,Ticker,GICS Sector,GICS Sub-Industry
0,Activision Blizzard,ATVI,Communication Services,Interactive Home Entertainment
1,Adobe,ADBE,Information Technology,Application Software
2,ADP,ADP,Information Technology,Data Processing & Outsourced Services
3,Airbnb,ABNB,Consumer Discretionary,Internet & Direct Marketing Retail
4,Align,ALGN,Health Care,Health Care Supplies
...,...,...,...,...
97,Workday,WDAY,Information Technology,Application Software
98,Xcel Energy,XEL,Utilities,Multi-Utilities
99,Xilinx,XLNX,Information Technology,Semiconductors
100,Zoom,ZM,Information Technology,Application Software


In [5]:
n100 = nasdaq_df[3]

In [6]:
n100

,Company,Ticker,GICS Sector,GICS Sub-Industry
0,Activision Blizzard,ATVI,Communication Services,Interactive Home Entertainment
1,Adobe,ADBE,Information Technology,Application Software
2,ADP,ADP,Information Technology,Data Processing & Outsourced Services
3,Airbnb,ABNB,Consumer Discretionary,Internet & Direct Marketing Retail
4,Align,ALGN,Health Care,Health Care Supplies
...,...,...,...,...
97,Workday,WDAY,Information Technology,Application Software
98,Xcel Energy,XEL,Utilities,Multi-Utilities
99,Xilinx,XLNX,Information Technology,Semiconductors
100,Zoom,ZM,Information Technology,Application Software


In [7]:
ticker_sec_gics = pd.DataFrame()
ticker_sec_gics = n100[['Ticker','Company','GICS Sector', 'GICS Sub-Industry']]

In [8]:
ticker_sec_gics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Ticker             102 non-null    object
 1   Company            102 non-null    object
 2   GICS Sector        102 non-null    object
 3   GICS Sub-Industry  102 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB


In [9]:
ticker = ticker_sec_gics['Ticker'].to_list()

In [10]:
start_date = '2019-03-31'
end_date = '2021-04-01'

In [11]:
main_df = pd.DataFrame()

In [12]:
len(ticker)

102

In [ ]:
for sym in ticker:

  temp_df = yf.download('{}'.format(sym), start=start_date, end=end_date)
  temp_df = temp_df.drop(columns=['Adj Close'], axis=1)

  temp_industry = ticker_sec_gics[ticker_sec_gics['Ticker'] == '{}'.format(sym)]['GICS Sector'].to_list()[0]
  temp_subindustry = ticker_sec_gics[ticker_sec_gics['Ticker'] == '{}'.format(sym)]['GICS Sub-Industry'].to_list()[0]

  temp_df.insert(0, 'Ticker', '{}'.format(sym))
  temp_df.insert(1, 'GICS Sector', '{}'.format(temp_industry))
  temp_df.insert(2, 'GICS Sub-Industry', '{}'.format(temp_subindustry))

  main_df = pd.concat([main_df, temp_df])

In [14]:
main_df

,Ticker,GICS Sector,GICS Sub-Industry,Open,High,Low,Close,Volume
Date,,,,,,,,
2019-04-01,ATVI,Communication Services,Interactive Home Entertainment,46.500000,47.310001,46.160000,47.119999,9233400.0
2019-04-02,ATVI,Communication Services,Interactive Home Entertainment,47.419998,47.779999,46.200001,47.459999,9697700.0
2019-04-03,ATVI,Communication Services,Interactive Home Entertainment,47.750000,48.599998,47.400002,47.650002,9741800.0
2019-04-04,ATVI,Communication Services,Interactive Home Entertainment,47.680000,47.820000,46.580002,46.959999,6941800.0
2019-04-05,ATVI,Communication Services,Interactive Home Entertainment,47.080002,47.540001,46.680000,47.290001,5976900.0
...,...,...,...,...,...,...,...,...
2021-03-25,ZS,Information Technology,Application Software,171.449997,175.300003,168.399994,172.850006,1557200.0
2021-03-26,ZS,Information Technology,Application Software,170.919998,174.000000,164.669998,169.779999,1816700.0
2021-03-29,ZS,Information Technology,Application Software,169.009995,174.860001,163.449997,165.750000,2372300.0


## Data Preparation

Code needs to::

iterate rows:

  -do basic calculation to determine presence of indicator

  -if indicator found, move on
  
  -save indicator in another dataframe with:

    -ticker, gics sec, gics sub-industry, indicator type, close, volume, uptrend/downtrend/neither, close value in 1d, 3d, 7d, 10d.

- check that close < open
- symbols are the same
- 2 days prior to target
- only want to check if there is indicator present

In [15]:
class Indicator:

  indicator_df = pd.DataFrame(columns=['Date', 'Ticker', 'GICS Sector', 'GICS Sub-Industry',
                                       'Close', 'Volume','Indicator_type', 'Indicator_trend', 'Trend', 
                                       '1d_pred', '3d_pred', '5d_pred', '10d_pred'])
  
  @staticmethod
  def indicator_detect(df: pd.DataFrame):
    
    for index in range(df.shape[0]):
      cur_row = df.iloc[index]
      cur_ticker, cur_open, cur_high, cur_low, cur_close = Indicator.assign_values(cur_row)
      Hammer.hammer_detect(df, index, cur_open, cur_high, cur_low, cur_close)
    
  def assign_values(row):
    """Extracts the ticker, open, high, low, and close values from a row"""
    return row[0], row[3], row[4], row[5], row[6]
  
  def validate_tickers(target_ticker: str, list_of_assigned_values: list):
    """Ensures that all"""
    for item in list_of_assigned_values:
      if item[0] != target_ticker:
        return False
    return True
  
  def add_entry(entry):
    Indicator.indicator_df.loc[len(Indicator.indicator_df)] = entry
    return True



In [16]:
class Trend:
  
  def downtrend(df, index):
    """Checks for the presence of a downtrend within 2 days of the indicator"""

    two_day_ago = (Indicator.assign_values(df.iloc[index - 2]))
    one_day_ago = (Indicator.assign_values(df.iloc[index - 1]))
    curr_day    = (Indicator.assign_values(df.iloc[index]))

    three_day = [two_day_ago, one_day_ago, curr_day]

    if Indicator.validate_tickers(curr_day[0], three_day) == False:
      return False

    for i in range(1):
      if three_day[i][4] < three_day[i + 1][4]:
        return False

    return True

current ticker against current predicted value

In [17]:
class Predict:

  def predictions(df, index):
    """Provides the close value with respect to
       the indicator 1, 3, 5,and 10 days ahead"""
    days = (1, 3, 5, 10)
    temp_list = []
    predicted_closing = []
    for day in days:
      temp_list.append(df.iloc[index + day])
      if Indicator.validate_tickers(df.iloc[index][0], temp_list) is False:
        temp_list.pop()
        break
    for item in temp_list:
        predicted_closing.append(item['Close'])
    Predict.fill_in_null_values(predicted_closing)
    return tuple(predicted_closing)

  def fill_in_null_values(list_of_tuples: list, desired_length=4):
    null_values_needed = 4 - len(list_of_tuples)
    for i in range(null_values_needed):
      list_of_tuples.append(None)
    return True

hamer criteria: 


*   high ~ open / close value
*   low > 2x the lower bound



In [18]:
class Hammer:
  """Identifies and creates a ammer entries in indicator_df"""

  def hammer_entry(df, index):
    """Creates the value of the hammer entry into indicator_df"""
    full_entry = []
    current_entry = df.iloc[index]
    full_entry = [df.index[index],
                  current_entry[0], 
                  current_entry[1],
                  current_entry[2],
                  current_entry[6],
                  current_entry[7],
                  'Hammer',
                  Hammer.hammer_trend(current_entry['Open'], current_entry['Close']),
                  Trend.downtrend(df, index)
                  ]
    predictions = Predict.predictions(df, index)
    for item in predictions:
      full_entry.append(item)
    return full_entry

  def hammer_criteria(open, high, low, close):
    """Tests for the criteria of te hammer indicator on a day"""
    upper_bound, lower_bound = max(open, close), min(open, close)
    check_one = Hammer.upper_wick_check(upper_bound, high)
    check_two = Hammer.lower_wick_check(upper_bound, lower_bound, low)
    if check_one == True and check_two == True:
      return True    


  def hammer_detect(df, index, open, high, low, close):
    """Detects the presence of a hammer and creates an entry if present"""
    crit_check = Hammer.hammer_criteria(open, high, low, close)
    if crit_check == True:
      indicator_entry = Hammer.hammer_entry(df, index)
      Indicator.add_entry(indicator_entry)
  

  def upper_wick_check(upper_bound, high):
    """Tests for the upper wick of the hammer. It should be approxmiately the 
       same as the daily high"""
    upper_range = [upper_bound * 0.999, upper_bound * 1.001]
    if high >= upper_range[0] and high <= upper_range[1]:
      return True

  def lower_wick_check(upper_bound, lower_bound, low):
    """Tests the lower wick of the hammer. It should be at least twice the size
       of the candle body"""
    day_diff = upper_bound - lower_bound
    if (lower_bound - low) >= (2 * day_diff):
      return True
  
  def hammer_trend(open, close):
    if open > close:
      return 'Bearish'
    else:
      return 'Bullish'

In [19]:
Indicator.indicator_detect(main_df)

In [20]:
Indicator.indicator_df = Indicator.indicator_df.round(2)

In [21]:
Indicator.indicator_df

,Date,Ticker,GICS Sector,GICS Sub-Industry,Close,Volume,Indicator_type,Indicator_trend,Trend,1d_pred,3d_pred,5d_pred,10d_pred
0,2019-07-05,ATVI,Communication Services,Interactive Home Entertainment,48.15,6379600.0,Hammer,Bearish,False,46.47,46.84,47.200001,45.139999
1,2019-07-22,ATVI,Communication Services,Interactive Home Entertainment,45.57,3771800.0,Hammer,Bullish,True,45.92,47.189999,48.450001,46.099998
2,2019-07-23,ATVI,Communication Services,Interactive Home Entertainment,45.92,3774300.0,Hammer,Bullish,False,46.86,47.990002,48.290001,46.59
3,2019-07-25,ATVI,Communication Services,Interactive Home Entertainment,47.19,4354000.0,Hammer,Bearish,False,47.99,48.290001,50.189999,49.330002
4,2019-08-22,ATVI,Communication Services,Interactive Home Entertainment,49.08,4001800.0,Hammer,Bearish,False,47.33,51.09,50.959999,54.580002
...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,2020-12-11,ZM,Information Technology,Application Software,397.01,4677400.0,Hammer,Bullish,False,395.99,404.0,406.01001,351.390015
874,2020-02-06,ZS,Information Technology,Application Software,58.40,1015300.0,Hammer,Bearish,True,58.97,60.919998,61.900002,54.509998
875,2020-05-27,ZS,Information Technology,Application Software,74.53,3720100.0,Hammer,Bearish,True,75.80,109.980003,107.410004,105.720001
876,2020-09-15,ZS,Information Technology,Application Software,132.64,1818900.0,Hammer,Bullish,False,131.06,129.100006,137.990005,140.490005


In [287]:
import os

In [ ]:
os.makedirs('csv_to_go')

In [288]:
Indicator.indicator_df.to_csv('csv_to_go/{}hammers.csv'.format)